In [21]:
import os
import langchain
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain import PromptTemplate, LLMChain
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.schema.runnable import RunnablePassthrough

from PyPDF2 import PdfReader
from helper_utils import reformulate_text
from helper_utils import word_wrap


In [8]:
HF_api_key = "....."

In [36]:
reader = PdfReader("dataset\The Hundred-Page Machine Learning Book.pdf")
page = reader.pages[100]
print(page.extract_text())

If the number of possible values for labels is high, but they are all of the same nature, like
tags, we can transform each labeled example into several labeled examples, one per label.
These new examples all have the same feature vector and only one label. That becomes a
multiclass classiﬁcation problem. We can solve it using the one versus rest strategy. The only
difference with the usual multiclass problem is that now we have a new hyperparameter:
threshold. If the prediction score for some label is above the threshold, this label is predicted
for the input feature vector. In this scenario, multiple labels can be predicted for one feature
vector. The value of the threshold is chosen using the validation set.
Analogously, algorithms that naturally can be made multiclass (decision trees, logistic regres-
sion and neural networks among others) can be applied to multi-label classiﬁcation problems.
Because they return the score for each class, we can deﬁne a threshold and then assign
mult

In [28]:
loader = PyPDFLoader("dataset\The Hundred-Page Machine Learning Book.pdf")

#pages = loader.load_and_split()
doc = loader.load()

In [52]:
# Split the document into chunks, embed each chunk and load it into the vector store.

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(doc)
db = Chroma.from_documents(documents, HuggingFaceEmbeddings())

# Connect query to chroma index using a retriever

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [48]:
query = "What is logistic regression?"
docs = db.similarity_search(query)
print(docs[0].page_content)


to a system of equations that gives us the optimal values w∗andb∗.
3.2 Logistic Regression
The ﬁrst thing to say is that logistic regression is not a regression, but a classiﬁcation learning
algorithm. The name comes from statistics and is due to the fact that the mathematical
formulation of logistic regression is similar to that of linear regression.
I explain logistic regression on the case of binary classiﬁcation. However, it can naturally be
extended to multiclass classiﬁcation.
3.2.1 Problem Statement
Inlogistic regression , we still want to model yias a linear function of xi, however, with a
binaryyithis is not straightforward. The linear combination of features such as wxi+bis a
function that spans from minus inﬁnity to plus inﬁnity, while yihas only two possible values.
At the time where the absence of computers required scientists to perform manual calculations,
they were eager to ﬁnd a linear classiﬁcation model. They ﬁgured out that if we deﬁne a
negative label as 0and the p

In [49]:
embedding_vector = HuggingFaceEmbeddings().embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

to a system of equations that gives us the optimal values w∗andb∗.
3.2 Logistic Regression
The ﬁrst thing to say is that logistic regression is not a regression, but a classiﬁcation learning
algorithm. The name comes from statistics and is due to the fact that the mathematical
formulation of logistic regression is similar to that of linear regression.
I explain logistic regression on the case of binary classiﬁcation. However, it can naturally be
extended to multiclass classiﬁcation.
3.2.1 Problem Statement
Inlogistic regression , we still want to model yias a linear function of xi, however, with a
binaryyithis is not straightforward. The linear combination of features such as wxi+bis a
function that spans from minus inﬁnity to plus inﬁnity, while yihas only two possible values.
At the time where the absence of computers required scientists to perform manual calculations,
they were eager to ﬁnd a linear classiﬁcation model. They ﬁgured out that if we deﬁne a
negative label as 0and the p

In [ ]:
# # Tokenizer

# model_name='mistralai/Mistral-7B-Instruct-v0.1'

# model_config = transformers.AutoConfig.from_pretrained(
#     model_name,
# )

# # Load pre-trained config

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
# )

# text_generation_pipeline = transformers.pipeline(
#     model=model,
#     tokenizer=tokenizer,
#     task="text-generation",
#     temperature=0.2,
#     repetition_penalty=1.1,
#     return_full_text=True,
#     max_new_tokens=300,
# )

#mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [9]:

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(repo_id=repo_id, 
                          max_new_tokens=512,
                          temperature=0.7, 
                           huggingfacehub_api_token= HF_api_key
)


c:\Users\moham\Desktop\Rag_Mistral\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\moham\.cache\huggingface\token
Login successful


In [55]:

template = """
Vous êtes un assistant IA serviable et vous fournissez la réponse à la question posée poliment.

{question}
Réponse : Réfléchissons étape par étape.
"""
# Create prompt from prompt template 
prompt = PromptTemplate(template=template, 
                        input_variables=["context","question"])

# Create llm chain 
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "quelles sont les monuments de la ville de toulouse?"


In [56]:
print(word_wrap(llm_chain.invoke(question)['text']))


La ville de Toulouse, en France, est reconnue pour son riche patrimoine
architectural. Voici une liste de quelques-uns des monuments les plus connus et
emblématiques :

1. La Basilique Saint-Sernin : C'est une basilique romane
construite au XIIe siècle. Elle est l'une des plus grandes églises romanes du
monde et est dédiée à Saint-Sernin, le saint patron de Toulouse.
2. La
Cathédrale Saint-Etienne : C'est une cathédrale gothique construite au XIVe
siècle. Elle est dédiée à saint Étienne, le premier martyr chrétien.
3. Le
Capitole : C'est un bâtiment historique situé au centre-ville de Toulouse. Il
abrite le Musée des Augustins et le théâtre du Capitole.
4. La Place du
Capitole : C'est la place centrale de Toulouse. Elle est entourée de belles
façades et de magnifiques bâtiments, dont le Capitole.
5. La Porte Narbonnaise
: C'est une porte fortifiée du XIIIe siècle qui était autrefois une des entrées
principales de la ville.
6. Le Pont Neuf : C'est le plus grand et le plus
ancien pont d

In [66]:
#RAG cchain

query = "c'est quoi la machine learning?" 

rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

rag_chain.invoke(query)

{'context': [Document(page_content='Chapter 1\nIntroduction\n1.1 What is Machine Learning\nMachine learning is a subﬁeld of computer science that is concerned with building algorithms\nwhich, to be useful, rely on a collection of examples of some phenomenon. These examples\ncan come from nature, be handcrafted by humans or generated by another algorithm.\nMachine learning can also be deﬁned as the process of solving a practical problem by 1)\ngathering a dataset, and 2) algorithmically building a statistical model based on that dataset.\nThat statistical model is assumed to be used somehow to solve the practical problem.\nTo save keystrokes, I use the terms “learning” and “machine learning” interchangeably.\n1.2 Types of Learning\nLearning can be supervised, semi-supervised, unsupervised and reinforcement.\n1.2.1 Supervised Learning\nInsupervised learning1, the dataset is the collection of labeled examples {(xi,yi)}N\ni=1.\nEach element xiamongNis called a feature vector . A feature ve

In [73]:
reformulated_output = reformulate_text(rag_chain.invoke(query)['text'])
print(reformulated_output)

La Machine Learning est un sous-ensemble d'Apprentissage Automatique
(Machine Intelligence), qui consiste à développer des modèles
informatiques capables d'améliorer leurs performances en apprenant
d'eux-mêmes à partir des données. Ces modèles utilisent des
algorithmes complexes pour découvrir des structures et des tendances
dans les données, et puis exploiter ces structures et tendances pour
résoudre des problèmes complexes, tels que la prédiction, la
classification ou la détection d'anomalies. Ainsi, Machine Learning
est un processus de développement de modèles informatiques qui peut
apprendre et améliorer ses performances automatiquement à partir des
données. Je vous espère que cela vous aide à comprendre ce concept! Si
vous avez d'autres questions ou si je peux vous fournir plus
d'informations, n'hésitez pas à me demander!
